New Project File:

Shell for the CAPTCHA Cracking Challenge

There are 8501 training images and a secret number of test images

In [1]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from matplotlib import patches as ptc 
import math
import random
from PIL import Image
import os
import re

# Define function for converting text to one-hot numerical encoding

def OneHotConverter(textstring, dictionary): # Needs to be a reverse dictionary

    ans = np.zeros((6, 21))
    for i in range(6):
        char = textstring[i].upper()
        num = dictionary[char]#dict2
        ans[i][num] = 1
    return ans 


    


# Define dictionaries for encoding/decoding
dict1 = {0:'2', 1:'6', 2:'A', 3:'B', 4:'C', 5:'D', 6:'E', 7:'F', 8:'G', 9:'H', 10:'K', 11:'M', 
              12:'N', 13:'P', 14:'R', 15:'S', 16:'T', 17:'U', 18:'V', 19:'X', 20:'Z'}
dict2 = {v: k for k, v in dict1.items()}



trainfiles2 = os.listdir('./traindata/')
trainfiles = [f for f in trainfiles2 if os.path.isfile('.' +'/traindata/' + f)] #Filtering only the files.
valfiles2 = os.listdir('./valdata/')
valfiles = [f for f in valfiles2 if os.path.isfile('.' +'/valdata/' + f)]




random.shuffle(trainfiles)
numtrain = len(trainfiles)
numval = len(valfiles)

train_images = np.zeros((numtrain,250,50))
train_labels = np.zeros((numtrain,6,21))
train_labels1 = np.zeros((numtrain, 21))
train_labels2 = np.zeros((numtrain, 21))
train_labels3 = np.zeros((numtrain, 21))
train_labels4 = np.zeros((numtrain, 21))
train_labels5 = np.zeros((numtrain, 21))
train_labels6 = np.zeros((numtrain, 21))
train_numlist = np.zeros((numtrain))
train_letters = ['     '] * numtrain

val_images = np.zeros((numval,250,50))
val_labels = np.zeros((numval,6,21))
val_labels1 = np.zeros((numval, 21))
val_labels2 = np.zeros((numval, 21))
val_labels3 = np.zeros((numval, 21))
val_labels4 = np.zeros((numval, 21))
val_labels5 = np.zeros((numval, 21))
val_labels6 = np.zeros((numval, 21))
val_numlist = np.zeros((numval))
val_letters = ['     '] * numval


# Define training set
for i in range(numtrain):
    img = Image.open('./traindata/' + trainfiles[i]).convert('L')
    data = np.transpose(np.asarray(img)) / 255
    train_images[i] = data
    train_numlist[i] = re.split(r"[_.]", trainfiles[i])[0]
    temp = re.split(r"[_.]", trainfiles[i])
    train_labels[i] = OneHotConverter(temp[1], dict2)
    train_labels1[i] = train_labels[i][0]
    train_labels2[i] = train_labels[i][1]
    train_labels3[i] = train_labels[i][2]
    train_labels4[i] = train_labels[i][3]
    train_labels5[i] = train_labels[i][4]
    train_labels6[i] = train_labels[i][5]
    train_letters[i] = temp[1].upper()

# Define validation set
for i in range(numval):
    img = Image.open('./valdata/' + valfiles[i]).convert('L')
    data = np.transpose(np.asarray(img)) / 255
    val_images[i] = data
    val_numlist[i] = re.split(r"[_.]", valfiles[i])[0]
    temp = re.split(r"[_.]", valfiles[i])
    val_labels[i] = OneHotConverter(temp[1], dict2)
    val_labels1[i] = val_labels[i][0]
    val_labels2[i] = val_labels[i][1]
    val_labels3[i] = val_labels[i][2]
    val_labels4[i] = val_labels[i][3]
    val_labels5[i] = val_labels[i][4]
    val_labels6[i] = val_labels[i][5]
    val_letters[i] = temp[1].upper()



# Note to self: add validation and test sets, comment out test set construction since the students
# Won't get the test set until the final




# Below is an example of how you can get the info from a train or validation file
print(re.split(r"[_.]", trainfiles[0]))

print(train_letters[0])
print(train_numlist[0])



['1590', 'maezxf', 'jpeg']
MAEZXF
1590.0


Define the model.  

Input layer: 250x50 pixel images in grayscale (0 to 1 values)

ADD NETWORK HERE


Final activation function needs to be softmax
Output: 6*21 matrix, each of the 6 21 entry rows is a confidence vector for the given character
Loss functions: 6 copies of cross-entropy for the 6 different characters


In [2]:


inputlayer = tf.keras.Input(shape=(250, 50, 1), name="Input")


# Right now the network is basically the same as our example McDigit one
# You'll need to add/change a lot between here and the dense layers at the end
convlayer1 = tf.keras.layers.Conv2D(filters=60, kernel_size=(4,4),padding='same', name="Convolution")(inputlayer)
maxpoollayer1 = tf.keras.layers.MaxPooling2D(pool_size=(4,4),strides=(3,3), name="MaxPool1")(convlayer1)


convlayer2 = tf.keras.layers.Conv2D(filters=50, kernel_size=(4,4),padding='same', name="Convolution2")(maxpoollayer1)
maxpoollayer2 = tf.keras.layers.MaxPooling2D(pool_size=(4,4),strides=(1,1), name="MaxPool2")(convlayer2)
dropout_layer1 = tf.keras.layers.Dropout(rate=0.25,name="Dropout1")(maxpoollayer2)

convlayer3 = tf.keras.layers.Conv2D(filters=50, kernel_size=(4,4),padding='same', name="Convolution3")(dropout_layer1)
maxpoollayer3 = tf.keras.layers.MaxPooling2D(pool_size=(3,3),strides=(1,1), name="MaxPool3")(convlayer3)

convlayer4 = tf.keras.layers.Conv2D(filters=20, kernel_size=(4,4),padding='same', name="Convolution4")(maxpoollayer3)
maxpoollayer4 = tf.keras.layers.MaxPooling2D(pool_size=(3,3),strides=(1,1), name="MaxPool4")(convlayer4)
denselayer1 = tf.keras.layers.Dense(40, activation='relu', name='Dense')(maxpoollayer4)

convlayer5 = tf.keras.layers.Conv2D(filters=10, kernel_size=(4,4),padding='same', name="Convolution5")(denselayer1)
maxpoollayer5 = tf.keras.layers.MaxPooling2D(pool_size=(3,3),strides=(1,1), name="MaxPool5")(convlayer5)
dropout_layer2 = tf.keras.layers.Dropout(rate=0.25,name="Dropout2")(maxpoollayer5)
convlayer6 = tf.keras.layers.Conv2D(filters=5, kernel_size=(4,4),padding='same', name="Convolution6")(dropout_layer2)
batchnorml = tf.keras.layers.BatchNormalization(name="Bathcnorm")(convlayer6)
maxpoollayer6 = tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(1,1), name="MaxPool6")(batchnorml)

flatlayer = tf.keras.layers.Flatten(name="Flatten")(maxpoollayer6)

# 







char1 = tf.keras.layers.Dense(21, activation='softmax', name='Char_1')(flatlayer)
char2 = tf.keras.layers.Dense(21, activation='softmax', name='Char_2')(flatlayer)
char3 = tf.keras.layers.Dense(21, activation='softmax', name='Char_3')(flatlayer)
char4 = tf.keras.layers.Dense(21, activation='softmax', name='Char_4')(flatlayer)
char5 = tf.keras.layers.Dense(21, activation='softmax', name='Char_5')(flatlayer)
char6 = tf.keras.layers.Dense(21, activation='softmax', name='Char_6')(flatlayer)

model = tf.keras.Model(inputs=inputlayer, outputs=[char1, char2, char3, char4, char5, char6])

model.summary()

model.compile(
  optimizer=tf.keras.optimizers.Adam(0.001),
  loss={
      "Char_1": tf.keras.losses.CategoricalCrossentropy(),
      "Char_2": tf.keras.losses.CategoricalCrossentropy(),
      "Char_3": tf.keras.losses.CategoricalCrossentropy(),
      "Char_4": tf.keras.losses.CategoricalCrossentropy(),
      "Char_5": tf.keras.losses.CategoricalCrossentropy(),
      "Char_6": tf.keras.losses.CategoricalCrossentropy()
      },
  metrics=['accuracy']
  )

tf.keras.utils.plot_model(model, "model.png")

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input (InputLayer)             [(None, 250, 50, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 Convolution (Conv2D)           (None, 250, 50, 60)  1020        ['Input[0][0]']                  
                                                                                                  
 MaxPool1 (MaxPooling2D)        (None, 83, 16, 60)   0           ['Convolution[0][0]']            
                                                                                                  
 Convolution2 (Conv2D)          (None, 83, 16, 50)   48050       ['MaxPool1[0][0]']           

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [100]:
batchsize = 128 # You may need to change this
numbatches = math.ceil(numtrain / batchsize)

checkpointfreq = 10 # Saves every this many epochs

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    "myweights1",
    save_freq=numbatches * checkpointfreq,
    save_weights_only=True)


model.fit(train_images, [train_labels1, train_labels2, train_labels3,
                        train_labels4, train_labels5, train_labels6],
                        validation_data=(val_images, [
                        val_labels1, val_labels2, val_labels3,
                        val_labels4, val_labels5, val_labels6
                        ]),
                        batch_size=batchsize,
                        epochs=50,
                        callbacks=[model_checkpoint_callback])





Epoch 1/50
67/67 [==============================] - 49s 725ms/step - loss: 19.5659 - Char_1_loss: 3.3059 - Char_2_loss: 3.2597 - Char_3_loss: 3.2442 - Char_4_loss: 3.2739 - Char_5_loss: 3.2449 - Char_6_loss: 3.2373 - Char_1_accuracy: 0.0488 - Char_2_accuracy: 0.0505 - Char_3_accuracy: 0.0475 - Char_4_accuracy: 0.0543 - Char_5_accuracy: 0.0551 - Char_6_accuracy: 0.0566 - val_loss: 18.5281 - val_Char_1_loss: 3.1205 - val_Char_2_loss: 3.1336 - val_Char_3_loss: 3.1255 - val_Char_4_loss: 3.0848 - val_Char_5_loss: 3.0346 - val_Char_6_loss: 3.0292 - val_Char_1_accuracy: 0.0740 - val_Char_2_accuracy: 0.0740 - val_Char_3_accuracy: 0.0800 - val_Char_4_accuracy: 0.0680 - val_Char_5_accuracy: 0.0800 - val_Char_6_accuracy: 0.0780
Epoch 2/50
67/67 [==============================] - 48s 723ms/step - loss: 15.6094 - Char_1_loss: 2.6068 - Char_2_loss: 2.6129 - Char_3_loss: 2.6259 - Char_4_loss: 2.6204 - Char_5_loss: 2.6374 - Char_6_loss: 2.5060 - Char_1_accuracy: 0.1713 - Char_2_accuracy: 0.1779 - Char

KeyboardInterrupt: 

In [38]:
#model.save_weights('myweights1') 
#Use this to save weights.  Weights are saved after every 10 epochs

In [3]:
model.load_weights('myweights3')
#Use this to load pre-trained weights or a checkpoint

Below here, put your function for taking the output of the model on a given image and returning a string.  Pass an image to the function.

For example, suppose we pass training image 0_vgxrub.jpeg to the model, and get an output of 6*21.  We pass this output to the function (along with the dictionary defined in the first cell), and it should return the string containing each of the six characters.  In this case, it would return VGXRUB.

In [4]:
# Example of how to pass a particular image to the model

# test1 = np.expand_dims(train_images[1],0)
# outtest1 = model.predict(test1)

# The first character's output, for example, is 

# print(outtest1[0][0])
# print(sum(outtest1[0][0]))
def OneHotDecoder(one_hot_matrix, dictionary):
    decoded_string = ""
    for row in one_hot_matrix:
        index = np.argmax(row)
        char = dictionary[index]
        decoded_string += char
    return decoded_string


for i in range(6):
    test_input = np.expand_dims(val_images[6], 0)
    output = model.predict(test_input)
    decoded_text = OneHotDecoder(output, dict1)

    print(f"Output for character {i + 1}:")
    print(output[i][0])
    print("Sum of output values:", sum(output[0][0]))
    print(f"Decoded text for character {i + 1}: {decoded_text[i]}")
    print("\n")


print(val_numlist[6])

# Your function should go here:


# def ConvertOutput(output, dictionary):
#     string = ""
#     max_val = 0
#     for i in range(6):
#         for j in range(21):
#             if output[i][j]>max_val:
#                 max_val = output[i][j]
#                 index = j
#         string += dictionary[index]
#     return string

# resultstring = ConvertOutput(outtest1[0], dict1)
# print(resultstring)




1/1 [==============================] - 0s 127ms/step
Output for character 1:
[7.6791013e-21 3.8838519e-10 2.9100948e-19 1.8411942e-19 3.8835966e-23
 7.6629131e-15 4.1912995e-21 6.5506061e-17 3.9305736e-31 7.2548809e-22
 7.1808675e-15 8.5266549e-13 3.6427517e-16 1.3982396e-20 1.7009596e-23
 6.8786297e-18 4.5062698e-16 7.0744541e-21 1.0000000e+00 3.8557719e-15
 6.3280235e-11]
Sum of output values: 1.0000000004525376
Decoded text for character 1: V


1/1 [==============================] - 0s 12ms/step
Output for character 2:
[1.08595123e-06 4.42587103e-16 6.86508049e-15 2.35730568e-09
 1.99305199e-12 9.99998927e-01 1.24147480e-11 1.35083989e-23
 2.94213668e-20 5.69182028e-18 1.19949275e-23 1.46288082e-16
 3.39201289e-14 4.99300796e-08 7.49314896e-19 1.22707529e-17
 6.21344254e-10 2.50374613e-12 1.12820120e-13 1.88804996e-19
 6.17692470e-19]
Sum of output values: 1.0000000004525376
Decoded text for character 2: D


1/1 [==============================] - 0s 10ms/step
Output for character 3:

2023-12-12 09:22:51.071805: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Output for character 4:
[1.4316541e-13 2.4704086e-17 9.1542673e-26 1.0000000e+00 9.9958074e-23
 4.6991196e-14 1.3447945e-16 3.7038178e-23 6.3210882e-22 1.2983001e-21
 2.8391041e-28 3.0138775e-19 1.7348851e-23 4.1018400e-12 4.0339934e-14
 3.4772772e-16 2.3785768e-27 3.5912777e-21 6.8039124e-26 7.0705893e-23
 4.8791071e-20]
Sum of output values: 1.0000000004525376
Decoded text for character 4: B


1/1 [==============================] - 0s 14ms/step
Output for character 5:
[1.1208225e-12 1.6314566e-19 2.8323763e-20 1.3793264e-28 1.0000000e+00
 2.6003611e-15 9.4560289e-19 7.6728797e-17 5.6873573e-10 4.1553303e-24
 3.2785842e-16 7.5535977e-34 7.6972258e-21 6.9886977e-22 3.2929981e-24
 6.1475364e-15 3.1316995e-13 2.8641657e-14 1.3650330e-18 1.4270282e-16
 8.4208411e-13]
Sum of output values: 1.0000000004525376
Decoded text for character 5: C


1/1 [==============================] - 0s 9ms/step
Output for character 6:
[9.9999988e-01 4.9210505e-28 6.9128085e-08 1.1440610e-19 7.3691747e-17
 6.1

Below here is stuff for testing the model.  Don't run this code.  Leave it commented out for now.

In [5]:
testfiles2 = os.listdir('./valdata/')
testfiles = [f for f in testfiles2 if os.path.isfile('.' +'/valdata/' + f)]

numtest = len(testfiles)

test_images = np.zeros((numtest,250,50))
test_labels = np.zeros((numtest,6,21))
test_labels1 = np.zeros((numtest, 21))
test_labels2 = np.zeros((numtest, 21))
test_labels3 = np.zeros((numtest, 21))
test_labels4 = np.zeros((numtest, 21))
test_labels5 = np.zeros((numtest, 21))
test_labels6 = np.zeros((numtest, 21))
test_numlist = np.zeros((numtest))
test_letters = ['     '] * numtest



for i in range(numtest):
    img = Image.open('./valdata/' + testfiles[i]).convert('L')
    data = np.transpose(np.asarray(img)) / 255
    test_images[i] = data
    test_numlist[i] = re.split(r"[_.]", testfiles[i])[0]
    temp = re.split(r"[_.]", testfiles[i])
    test_labels[i] = OneHotConverter(temp[1], dict2)
    test_labels1[i] = test_labels[i][0]
    test_labels2[i] = test_labels[i][1]
    test_labels3[i] = test_labels[i][2]
    test_labels4[i] = test_labels[i][3]
    test_labels5[i] = test_labels[i][4]
    test_labels6[i] = test_labels[i][5]
    test_letters[i] = temp[1].upper()

In [6]:
results = model.evaluate(test_images, [test_labels1, test_labels2, test_labels3,
                        test_labels4, test_labels5, test_labels6])
avgaccuracy = 100*sum(results[-6:])/6 


print(f"Average character accuracy on test set: {avgaccuracy:.2f} percent")

16/16 [==============================] - 1s 53ms/step - loss: 0.7550 - Char_1_loss: 0.0854 - Char_2_loss: 0.1089 - Char_3_loss: 0.2018 - Char_4_loss: 0.1424 - Char_5_loss: 0.1918 - Char_6_loss: 0.0247 - Char_1_accuracy: 0.9780 - Char_2_accuracy: 0.9800 - Char_3_accuracy: 0.9640 - Char_4_accuracy: 0.9680 - Char_5_accuracy: 0.9760 - Char_6_accuracy: 0.9920
Average character accuracy on test set: 97.63 percent


In [4]:
testfiles2 = os.listdir('./testdata/')
testfiles = [f for f in testfiles2 if os.path.isfile('.' +'/testdata/' + f)]

numtest = len(testfiles)

test_images = np.zeros((numtest,250,50))
test_labels = np.zeros((numtest,6,21))
test_labels1 = np.zeros((numtest, 21))
test_labels2 = np.zeros((numtest, 21))
test_labels3 = np.zeros((numtest, 21))
test_labels4 = np.zeros((numtest, 21))
test_labels5 = np.zeros((numtest, 21))
test_labels6 = np.zeros((numtest, 21))
test_numlist = np.zeros((numtest))
test_letters = ['     '] * numtest



for i in range(numtest):
    img = Image.open('./testdata/' + testfiles[i]).convert('L')
    data = np.transpose(np.asarray(img)) / 255
    test_images[i] = data
    test_numlist[i] = re.split(r"[_.]", testfiles[i])[0]
    temp = re.split(r"[_.]", testfiles[i])
    test_labels[i] = OneHotConverter(temp[1], dict2)
    test_labels1[i] = test_labels[i][0]
    test_labels2[i] = test_labels[i][1]
    test_labels3[i] = test_labels[i][2]
    test_labels4[i] = test_labels[i][3]
    test_labels5[i] = test_labels[i][4]
    test_labels6[i] = test_labels[i][5]
    test_letters[i] = temp[1].upper()


                   

In [10]:
results = model.evaluate(test_images, [test_labels1, test_labels2, test_labels3,
                        test_labels4, test_labels5, test_labels6])
avgaccuracy = 100*sum(results[-6:])/6 


print(f"Average character accuracy on test set: {avgaccuracy:.2f} percent") 

32/32 [==============================] - 2s 55ms/step - loss: 0.8041 - Char_1_loss: 0.0716 - Char_2_loss: 0.1574 - Char_3_loss: 0.1870 - Char_4_loss: 0.1488 - Char_5_loss: 0.1757 - Char_6_loss: 0.0636 - Char_1_accuracy: 0.9780 - Char_2_accuracy: 0.9660 - Char_3_accuracy: 0.9720 - Char_4_accuracy: 0.9680 - Char_5_accuracy: 0.9590 - Char_6_accuracy: 0.9830
Average character accuracy on test set: 97.10 percent


In [5]:
def ConvertOutput(output, dictionary):
    # Code goes here
    string = ""
    for i in range(6):
      num = np.argmax(output[i])
      char = dict1[num]
      string += char

    return string

total_inacc = 0 
for index in range(1000):
#   test1 = np.expand_dims(train_images[index],0)
#   outtest1 = model.predict(test1)

#   test2 = np.expand_dims(val_images[index], 0)
#   outtest2 = model.predict(test2)
  test3 = np.expand_dims(test_images[index], 0)
  outtest3 = model.predict(test3)
    
  # The first character's output, for example, is  

  # Look at which entry has the largest value (probability).     
  #   train_image = ConvertOutput(outtest1, dict1)
  #   val_image = ConvertOutput(outtest2, dict1)
  test_image = ConvertOutput(outtest3, dict1)
  # train_image_true = train_letters[index]
  # val_image_true =  val_letters[index]
  test_image_true = test_letters[index]
  
  if test_image != test_image_true :
    print(test_image)
    print(test_image_true) 
    total_inacc += 1
    
print(total_inacc)

1/1 [==============================] - 0s 10ms/step


2023-12-13 23:32:55.458341: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 0s 10ms/step
GSEPDX
GSEDDX
1/1 [==============================] - 0s 9ms/step
URNDNN
URNSNN
1/1 [==============================] - 0s 11ms/step
CSFBHA
CSEBHA
1/1 [==============================] - 0s 9ms/step
BTSSSV
BTSSSB
1/1 [==============================] - 0s 10ms/step
TGBMMP
TGBMMR
1/1 [==============================] - 0s 12ms/step
2HEEDT
2HFEDT
1/1 [==============================] - 0s 11ms/step
ENCVDX
ENCVDZ
1/1 [==============================] - 0s 10ms/step
R66KVS
RV6KVS
1/1 [==============================] - 0s 9ms/step
6UUFEE
6GUGHE
1/1 [==============================] - 0s 10ms/step
ZCDXSV
ZGDXSV
1/1 [==============================] - 0s 10ms/step
VCXZRR
VCKZRR
1/1 [==============================] - 0s 10ms/step
226HAA
226HXA
1/1 [==============================] - 0s 11ms/step
MGKKBH
MGKKBE
1/1 [==============================] - 0s 12ms/step
DXUVX2
DXUTX2
1/1 [==============================] - 0s 11ms/step
VHNNVV
VHNANV
1/1 [========

1/1 [==============================] - 0s 9ms/step
6VGMZZ
EVGBCZ
1/1 [==============================] - 0s 10ms/step
DDVT2G
UDVT2G
1/1 [==============================] - 0s 11ms/step
CV6PHB
CVGPHB
1/1 [==============================] - 0s 10ms/step
2DGCCR
2DGGCR
1/1 [==============================] - 0s 10ms/step
ECU2CM
ECU2GM
1/1 [==============================] - 0s 11ms/step
CD6DUG
GD6DUG
1/1 [==============================] - 0s 10ms/step
HNDGTC
HNDGTG
1/1 [==============================] - 0s 10ms/step
NBBAUU
MVBAUU
1/1 [==============================] - 0s 11ms/step
PXVHZT
PXVRZT
1/1 [==============================] - 0s 10ms/step
TSNDUB
TSADUB
1/1 [==============================] - 0s 10ms/step
S6U2BZ
SGU2BZ
1/1 [==============================] - 0s 10ms/step
NNUHMV
NNUHMN
1/1 [==============================] - 0s 11ms/step
UGFMVK
UGTMVK
1/1 [==============================] - 0s 10ms/step
TST662
TST6K2
1/1 [==============================] - 0s 11ms/step
6RFEUR
6RFFUR
1/1 [======

1/1 [==============================] - 0s 10ms/step
UZMCCR
UZMGCR
1/1 [==============================] - 0s 10ms/step
RNNBMA
RUNBMA
1/1 [==============================] - 0s 10ms/step
CGRZU6
CGBZU6
1/1 [==============================] - 0s 10ms/step
RMMVVH
RMCVMH
1/1 [==============================] - 0s 11ms/step
TG6KK2
TG6KR2
1/1 [==============================] - 0s 10ms/step
CUXSHF
CUXSFF
1/1 [==============================] - 0s 10ms/step
SGZS6G
SG2S6G
1/1 [==============================] - 0s 11ms/step
XFVPVA
XFVPRA
1/1 [==============================] - 0s 11ms/step
6VFKAV
6VFRAV
1/1 [==============================] - 0s 10ms/step
EMMFCE
ETMFCE
1/1 [==============================] - 0s 10ms/step
ERXEK2
ERXFX2
1/1 [==============================] - 0s 9ms/step
CDXRHR
CDXKHR
1/1 [==============================] - 0s 10ms/step
22PKFZ
Z2PXFZ
1/1 [==============================] - 0s 10ms/step
SXSM6T
SXBM6T
1/1 [==============================] - 0s 11ms/step
C6CETC
G6CETC
1/1 [======

1/1 [==============================] - 0s 10ms/step
VXKCRT
VXHCRT
1/1 [==============================] - 0s 10ms/step
VXFAMT
NXFAMT
1/1 [==============================] - 0s 10ms/step
2PHKFT
ZPHKFT
1/1 [==============================] - 0s 10ms/step
RMHSFD
RMHSTD
1/1 [==============================] - 0s 10ms/step
6GZPVP
6GGPVP
1/1 [==============================] - 0s 10ms/step
TGZCZT
TGZGZT
1/1 [==============================] - 0s 11ms/step
2AFGMA
2AFGMX
1/1 [==============================] - 0s 11ms/step
AEABZ6
AEADZ6
1/1 [==============================] - 0s 10ms/step
6RGAMV
6RGAMX
1/1 [==============================] - 0s 10ms/step
ZTVCDM
ZTTCDM
1/1 [==============================] - 0s 10ms/step
REXT6D
REXTKD
1/1 [==============================] - 0s 10ms/step
FEMMAD
FSMMAD
1/1 [==============================] - 0s 10ms/step


1/1 [==============================] - 0s 11ms/step
GGGCCZ
RGGCCZ
1/1 [==============================] - 0s 10ms/step
TZKGHP
TZTGHP
1/1 [==============================] - 0s 10ms/step
BPAGEU
BPA6EU
1/1 [==============================] - 0s 10ms/step
TVAVCB
TVHVCB
1/1 [==============================] - 0s 10ms/step
2FN2CM
2FN2UM
1/1 [==============================] - 0s 11ms/step
2PC6UX
2PC6GX
1/1 [==============================] - 0s 10ms/step
KNRCFK
KNRCHK
1/1 [==============================] - 0s 10ms/step
PM2DBN
PM2DBK
1/1 [==============================] - 0s 10ms/step
CCFTHD
CCFTFD
1/1 [==============================] - 0s 10ms/step
EXF2PA
EXF22A
1/1 [==============================] - 0s 11ms/step
DSUH6D
DSDH6D
1/1 [==============================] - 0s 11ms/step
FFPFNT
FPDFNT
1/1 [==============================] - 0s 10ms/step
TTK6NM
TTKGNM
1/1 [==============================] - 0s 11ms/step
NCDUEV
NCBUEV
1/1 [==============================] - 0s 10ms/step
MKCSZZ
MKGSZZ
1/1 [=====

1/1 [==============================] - 0s 10ms/step
KKVDBS
KKADBS
1/1 [==============================] - 0s 11ms/step
V6DUPA
V6DUPD
1/1 [==============================] - 0s 10ms/step
M66FXH
KBCRXH
1/1 [==============================] - 0s 10ms/step
MCD6ZH
MGD6ZH
1/1 [==============================] - 0s 10ms/step
GMUDK2
GMUXK2
1/1 [==============================] - 0s 11ms/step
AVFZNP
AVFZKP
1/1 [==============================] - 0s 11ms/step
NEPMAD
NEPMAP
1/1 [==============================] - 0s 10ms/step
MF2GMM
MF2GUM
1/1 [==============================] - 0s 11ms/step
AAAATV
HAUATV
1/1 [==============================] - 0s 11ms/step
RRXMBZ
RVXMBZ
1/1 [==============================] - 0s 11ms/step
ZF62AB
ZH62AB
1/1 [==============================] - 0s 11ms/step
EEH2PM
BEHSPM
1/1 [==============================] - 0s 11ms/step
EPUDDX
EPUDBX
1/1 [==============================] - 0s 11ms/step
VR6TUT
VR6TUH
1/1 [==============================] - 0s 10ms/step
BXSPPM
BXGPPM
1/1 [=====

1/1 [==============================] - 0s 10ms/step
TFEFNU
RFEFNU
1/1 [==============================] - 0s 10ms/step
KDRXKN
KDRXEN
1/1 [==============================] - 0s 10ms/step
VVXK2B
KVXRDB
1/1 [==============================] - 0s 11ms/step
DTKHCZ
UTKHCZ
1/1 [==============================] - 0s 10ms/step
SFXCEU
SFX6EU
1/1 [==============================] - 0s 10ms/step
MRTR6A
MRTRDA
1/1 [==============================] - 0s 10ms/step
AUZA2F
AUZM2F
1/1 [==============================] - 0s 10ms/step
UBKFRM
UBKFPM
1/1 [==============================] - 0s 10ms/step
DDCP2H
DXUP2H
1/1 [==============================] - 0s 10ms/step
2ZDA6V
2ZDA6N
1/1 [==============================] - 0s 10ms/step
UE6CPK
UE6CFK
1/1 [==============================] - 0s 11ms/step
145


In [ ]:
print(100 - total_inacc/500*100)